In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from muarch import MUArch, UArch
from muarch.datasets import load_etf
from copulae import TCopula
import time
import datetime

C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [24]:
start = '2000-01-03'
end = '2005-12-31'


# Tickers of stocks - Inputted from dashboard
stocks = ['BAC', 'C', 'SNV', 'STI', 'WFC', 'LNC', 'PGR', 'GL', 'GS',
                  'SCHW', 'AXP', 'BEN', 'BLK', 'COF', 'MS']
stocks.sort()


#With SPY
assets = stocks.copy()
assets.append('SPY')

   
# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets


# Calculating returns
returns = data.pct_change().dropna()
# Return_Data.head()

[*********************100%***********************]  16 of 16 completed


In [ ]:
# def CoSR_Garch_Copula():
#     process_start_time = time.time()
#     print (time.strftime('%H:%M:%S',time.localtime(time.time())))

#     num_assets = returns.shape[1]

#     models = MUArch(num_assets, mean='AR', lags=1) 
#     models[0] = UArch('AR', lags=1, dist='skewt')  

#     models.fit(returns)

#     residuals = models.residuals() # defaults to return the standardized residuals
#     cop = TCopula(dim=num_assets)
#     cop.fit(residuals)

#     m = models.simulate_mc(30000, 1, custom_dist=cop.random)
#     process_stop_time = time.time()
#     diff_time = process_stop_time - process_start_time
#     print(diff_time) 
    
#     a = m
#     sim_data = []
#     for i in range(0,len(a)):
#         sim_data.append(list(a[i][0]))
        
#     simulation_data = pd.DataFrame(data = sim_data, columns = assets)
#     simulation_data.to_csv('garch_copula_simdata')
# ############################## SR ##########################################################    
#     tickers = stocks

#     Y = simulation_data[tickers]
#     port = pf.Portfolio(returns=Y)

#     method_mu='hist' 
#     method_cov='hist' 

#     port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
#     model='Classic' 
#     rm = 'MV' 
#     obj = 'Sharpe' 
#     hist = True 
#     rf = 0 
#     l = 0 

#     weights_SR = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
#     weights_SR.to_csv('weights_SR.csv')
# ################################ CoSR ##########################################################    
#     c = 0.0
#     num_portfolios = 2000
#     port_CoSR_ratio = []
#     port_volatility = []
#     stock_weights = []
#     weights = []

#     df = simulation_data
#     Y_SPY = pd.DataFrame(df[df['SPY']< c]['SPY'])
#     Y = df[df['SPY']< c][stocks] 

#     cov_matrix = Y.cov()
#     mean_returns = Y.mean()
#     mkt_returns = Y_SPY.SPY.mean()

#     for i in range(num_portfolios):
#         weights = np.random.random(len(stocks))
#         weights /= np.sum(weights)
#         mkt_returns = df['SPY'].mean()
#         returns = np.dot(weights, mean_returns)
#         volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
#         CoSR_ratio = (returns - mkt_returns)/volatility
#         port_CoSR_ratio.append(CoSR_ratio)
#         stock_weights.append(weights)

#     weight = stock_weights[port_CoSR_ratio.index(max(port_CoSR_ratio))]

#     weights_CoSR = pd.DataFrame(data = weight, index = stocks).T
#     return weights_SR,weights_CoSR

In [3]:
num_assets = returns.shape[1]
# sets up a MUArch model collection where each model defaults to
# mean: AR(1)
# vol: GARCH(1, 1)
# dist: normal
models = MUArch(num_assets, mean='AR', lags=1)

In [4]:
models[0] = UArch('AR', lags=1, dist='skewt')

In [5]:
models.fit(returns)


C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0008164. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scale_warning.format(orig_scale, rescale), DataScaleWarning
C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0007529. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=Fal

In [6]:
models.summary()

<class 'muarch.summary.SummaryList'>
"""
AXP

                                 AR - GARCH Model Results                                
=========================================================================================
Dep. Variable:                                 y   R-squared:                       0.000
Mean Model:                                   AR   Adj. R-squared:                 -0.003
Vol Model:                                 GARCH   Log-Likelihood:                787.213
Distribution:      Standardized Skew Student's t   AIC:                          -1560.43
Method:                       Maximum Likelihood   BIC:                          -1533.09
                                                   No. Observations:                  367
Date:                           Wed, Aug 26 2020   Df Residuals:                      360
Time:                                   02:37:57   Df Model:                            7
                                  Mean Model                                  
==============================================================================
                  coef    std err          t      P>|t|       95.0% Conf. Int.
------------------------------------------------------------------------------
Const      -2.1873e-04  1.391e-03     -0.157      0.875 [-2.946e-03,2.508e-03]
y[1]       -5.4811e-03  5.431e-02     -0.101      0.920      [ -0.112,  0.101]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      5.5742e-05  1.228e-05      4.539  5.663e-06 [3.167e-05,7.981e-05]
alpha[1]       0.0547  2.751e-02      1.990  4.663e-02   [8.160e-04,  0.109]
beta[1]        0.8779  2.180e-02     40.269      0.000     [  0.835,  0.921]
                                 Distribution                                
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
nu            56.9315    128.773      0.442      0.658 [-1.955e+02,3.093e+02]
lambda     3.3432e-03  7.258e-02  4.606e-02      0.963      [ -0.139,  0.146]
=============================================================================

Covariance estimator: robust

****************************************************************************************************

BAC

                           AR - GARCH Model Results                           
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Mean Model:                        AR   Adj. R-squared:                 -0.003
Vol Model:                      GARCH   Log-Likelihood:                803.420
Distribution:                  Normal   AIC:                          -1596.84
Method:            Maximum Likelihood   BIC:                          -1577.31
                                        No. Observations:                  367
Date:                Wed, Aug 26 2020   Df Residuals:                      362
Time:                        02:37:58   Df Model:                            5
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
Const      1.7042e-03  1.365e-03      1.249      0.212 [-9.704e-04,4.379e-03]
y[1]           0.0271  5.664e-02      0.479      0.632   [-8.389e-02,  0.138]
                              Volatility Model                              
======================

In [13]:
residuals = models.residuals() # defaults to return the standardized residuals
cop = TCopula(dim=num_assets)
cop.fit(residuals)
print(cop.summary())

Student Copula Summary
Student Copula with 16 dimensions


Parameters
--------------------------------------------------------------------------------
 Degree of Freedom  : 18.654661216217583

 Correlation Matrix 
 1.000000  0.616915  0.566181  0.190125  0.696778  0.618357  0.424638  0.504990  0.457978  0.562673  0.392641  0.556373  0.595013  0.576213  0.596964  0.578139
 0.616915  1.000000  0.535355  0.177361  0.695829  0.551012  0.427974  0.434816  0.468738  0.512396  0.411666  0.423783  0.659772  0.495584  0.712758  0.616553
 0.566181  0.535355  1.000000  0.264375  0.565061  0.547801  0.517512  0.448697  0.479970  0.521687  0.359566  0.498306  0.577868  0.528453  0.512581  0.457892
 0.190125  0.177361  0.264375  1.000000  0.272284  0.225661  0.193961  0.225585  0.176721  0.225106  0.107093  0.215969  0.207411  0.218594  0.159609  0.125860
 0.696778  0.695829  0.565061  0.272284  1.000000  0.605896  0.436460  0.611448  0.474020  0.653222  0.445385  0.577961  0.608999  0.629954  0.605

In [14]:
cop[:] = returns.corr()
print(cop.summary())

Student Copula Summary
Student Copula with 16 dimensions


Parameters
--------------------------------------------------------------------------------
 Degree of Freedom  : 18.654661216217583

 Correlation Matrix 
 1.000000  0.617836  0.573591  0.189864  0.706887  0.622784  0.389462  0.503187  0.402701  0.571734  0.343945  0.555084  0.592150  0.593235  0.591167  0.582923
 0.617836  1.000000  0.531350  0.149621  0.710325  0.567011  0.385047  0.451004  0.421965  0.521699  0.380695  0.425829  0.662012  0.513222  0.725966  0.626117
 0.573591  0.531350  1.000000  0.227719  0.572273  0.546086  0.463856  0.446838  0.472306  0.516619  0.342605  0.502282  0.577431  0.521320  0.526582  0.488377
 0.189864  0.149621  0.227719  1.000000  0.268382  0.202954  0.188536  0.230290  0.130828  0.232800  0.086214  0.196052  0.164708  0.212832  0.127477  0.119944
 0.706887  0.710325  0.572273  0.268382  1.000000  0.611999  0.379892  0.621380  0.415643  0.661500  0.413306  0.572364  0.604485  0.647726  0.622

In [25]:
returns.shape

(1508, 16)

In [29]:

process_start_time = time.time()
print (time.strftime('%H:%M:%S',time.localtime(time.time())))

# returns = load_etf()  # load returns data
num_assets = returns.shape[1]

# sets up a MUArch model collection where each model defaults to 
# mean: AR(1)
# vol: GARCH(1, 1)
# dist: normal 
models = MUArch(num_assets, mean='AR', lags=1) 

# set first model to AR(1)-GARCH(1, 1) with skewt innovations  
models[0] = UArch('AR', lags=1, dist='skewt')  

# fit model, if you get complaints regarding non-convergence, you can scale the data up 
# using the scale parameter in the UArch or MUArch. i.e. UArch(..., scale=100). This will
# reduce numerical errors. Don't worry, I'll rescale the simulation values subsequently
models.fit(returns)

# Usually you'll want to fit the residuals to the copula, use the copula to generate the
# residuals and subsequently transform it back to returns 

residuals = models.residuals() # defaults to return the standardized residuals

# residuals = models.residuals() # defaults to return the standardized residuals
cop = TCopula(dim=num_assets)
cop.fit(residuals)
# cop = NormalCopula(dim=num_assets) # use a normal copula, you could of course use a TCopula
# cop.fit(residuals)

# simulate 10 steps into the future, over 4 repetitions. This will return a (10 x 4 x 3) array
m = models.simulate_mc(30000, 1, custom_dist=cop.random)
process_stop_time = time.time()
diff_time = process_stop_time - process_start_time
print(diff_time) 
#3949.5193169116974 seconds

11:03:08


C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004679. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scale_warning.format(orig_scale, rescale), DataScaleWarning
C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003312. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=Fal

3949.5193169116974


In [30]:
m = models.simulate_mc(30000, 1, custom_dist=cop.random)

In [48]:
a = m
sim_data = []
for i in range(0,len(a)):
    sim_data.append(list(a[i][0]))

In [52]:
simulation_data = pd.DataFrame(data = sim_data, columns = assets)

In [54]:
simulation_data.to_csv('garch_copula_simdata')

In [55]:
simulation_data

,AXP,BAC,BEN,BLK,C,COF,GL,GS,LNC,MS,PGR,SCHW,SNV,STI,WFC,SPY
0,0.008182,0.011345,-0.014547,-0.013557,-0.002607,0.012036,-0.012253,-0.010084,-0.005235,-0.008686,0.018461,-0.006756,0.010206,0.007559,0.006265,0.012948
1,-0.013436,-0.001027,-0.023329,0.000008,-0.009003,-0.002605,-0.007810,-0.024782,-0.025063,-0.020174,-0.009891,-0.016782,-0.030422,-0.005813,-0.005318,0.005532
2,0.014228,-0.001357,0.003488,-0.008199,-0.006599,0.005440,-0.014663,-0.017553,-0.001540,-0.004339,0.001365,0.009658,-0.012552,-0.008611,-0.011446,0.011363
3,-0.018707,-0.013184,-0.004487,-0.003842,-0.001477,-0.014376,0.005448,-0.008996,-0.015497,-0.005405,-0.016759,-0.013115,-0.004998,-0.003990,-0.009844,0.004627
4,-0.009394,-0.029898,-0.010977,0.011465,-0.021181,-0.003216,-0.028210,-0.032665,-0.040160,-0.006835,-0.022222,-0.000489,-0.038890,-0.017317,-0.014992,-0.006163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.014555,0.002623,0.003558,0.003270,0.015313,0.019882,-0.005940,0.000107,-0.000039,0.003964,0.000156,0.022606,0.010117,0.004615,0.003646,-0.000038
29996,-0.014255,0.004112,-0.001613,0.010905,-0.010634,0.006962,0.004410,-0.016169,-0.005451,-0.013201,-0.001605,-0.023666,0.004992,-0.003858,0.000909,-0.003388
29997,0.013979,0.007847,-0.009759,0.025983,-0.006579,-0.020773,0.008153,0.005432,-0.005052,0.005122,-0.008413,0.007260,0.001817,-0.000911,0.006502,-0.002700
29998,0.006720,0.008634,0.010840,-0.003185,0.010418,0.015441,-0.001646,0.001362,0.018874,0.001223,0.002540,0.027124,0.003500,0.002721,0.005125,0.004308


In [57]:
import matplotlib.pyplot as plt
import riskfolio.Portfolio as pf


import empyrical.stats as em
import quantstats as qs

C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\empyrical\utils.py:32: UserWarning: Unable to import pandas_datareader. Suppressing import error and continuing. All data reading functionality will raise errors; but has been deprecated and will be removed in a later version.
  warnings.warn(msg)


In [58]:
# SR weights
tickers = stocks
Y = simulation_data[tickers]

# Building the portfolio object
port = pf.Portfolio(returns=Y)
# Calculating optimum portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Configuring short weights options

#     port.sht = True # Allows to use Short Weights
#     port.uppersht = 0.3 # Maximum value of sum of short weights in absolute value

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

C:\Users\JIWU5\AppData\Local\Continuum\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


In [66]:
w.to_csv('weights_SR.csv')

In [63]:
# CoSR weights
c = 0.0
num_portfolios = 2000
port_CoSR_ratio = []
port_volatility = []
stock_weights = []
weights = []

df = simulation_data
Y_SPY = pd.DataFrame(df[df['SPY']< c]['SPY'])
Y = df[df['SPY']< c][stocks] 


cov_matrix = Y.cov()
mean_returns = Y.mean()
mkt_returns = Y_SPY.SPY.mean()


for i in range(num_portfolios):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    mkt_returns = df['SPY'].mean()
    returns = np.dot(weights, mean_returns)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    CoSR_ratio = (returns - mkt_returns)/volatility
    port_CoSR_ratio.append(CoSR_ratio)
    stock_weights.append(weights)

weight = stock_weights[port_CoSR_ratio.index(max(port_CoSR_ratio))]

df = pd.DataFrame(data = weight, index = stocks).T

In [67]:
Cosr = df.T

In [69]:
Cosr.to_csv('weights_CoSR.csv')